In this notebook, we will use the historical calendar entries from Florence to map to half-day number to identify the dates that those lessons will likely be taught in Hopper. Then put the entries in a generic form associated with Hopper dates to write to a csv that can be imported into a new calendar in google calendar. 

In [1]:
from csv_ical import Convert
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import cohort_calendar

In [2]:
# cohort to base the calendar off of
cohort = 'florence'

# read the export of florence calendar that is stored in working directory
flo_df = cohort_calendar.read_calendar_csv('florence')

# get the class days of florence
class_days_df = cohort_calendar.get_class_dates('Florence')

# clean the old calendar entries and get lesson details
flo_df = cohort_calendar.clean_cal(flo_df)
flo_df = cohort_calendar.extract_lesson_details(flo_df)

# merge the class days with the calendar details on start date and day segment (1 or 2 - half days)
flo_df = class_days_df.merge(flo_df, how='left', on=['start', 'day_segment']).drop(columns=('start'))

In [ ]:
# get the class days from hopper
hopper_class_days = cohort_calendar.get_class_dates('Hopper')

In [26]:
# concatenate hopper class days with florence class days by 1/2 day number on get the lessons that were taught
# on those days. 
cal = pd.concat([flo_df, hopper_class_days], axis=1)
cal = cal[['event_name', 'start']]
cal.iloc[201, 0] = 'Hopper Graduation'

In [27]:
# clean up the calendar before exporting...fill in these with capstone
fill_in = [191, 193, 195, 198]
for i in fill_in:
    cal.iloc[i, 0] = 'Capstone'

In [28]:
# drop rows with no event entries
cal = cal[cal.event_name.notnull()]

# drop the last row, that is past hopper graduation
cal = cal.iloc[:-1]

# rename columns so that google calendar will know how to import the csv into the calendar. 
cal.columns = ['Subject', 'Start Date']

In [29]:
# write calendar entries to csv. 
cal.to_csv('hopper_cal.csv')

Final step is to create "Hopper (instructors)" calendar in google calendar 
and import "hopper_cal.csv" to the calendar. 